<a href="https://colab.research.google.com/github/mnocerino23/Numerical-Analysis/blob/main/HillbertMatrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The function below will take in parameter size and create a size x size matrix with entries such that Hn(i,j) = 1/(i+j-1)

In [88]:
import numpy as np
import numpy.linalg as nplinalg
import scipy
import scipy.linalg

In [89]:
def create_hillbert(size):
  #initialize to a nxn matrix with 0's as entries
  matrix = np.zeros(shape = (size,size))
  row_count = 1
  for row in matrix:
    for i in range(len(row)):
      #index represents column count
      index = i+1
      row[i] = 1/(row_count+index-1)
    row_count += 1
  #calculate the vector containing the matrix row sums using np.sum(axis = 1)
  b = matrix.sum(axis=1, dtype='float')
  return (matrix, b)

Running the code to create a 4x4 Hillbert matrix and the associated b vector. The function returns a pair with the first element being the Hillbert matrix and the second element being the associated b vector with the row sums

In [90]:
create_hillbert(4)

(array([[1.        , 0.5       , 0.33333333, 0.25      ],
        [0.5       , 0.33333333, 0.25      , 0.2       ],
        [0.33333333, 0.25      , 0.2       , 0.16666667],
        [0.25      , 0.2       , 0.16666667, 0.14285714]]),
 array([2.08333333, 1.28333333, 0.95      , 0.75952381]))

Now solve for the solution

In [91]:
n_values = [5, 10, 15, 20]

In [92]:
import pandas as pd

Note the way we have coded our function
H[0] = Hillbert matrix
H[1] = b vector

In [93]:
#write a function to get LU decomposition of Hillbert matrix
def get_Hillbert_LU(H):
  P, L, U = scipy.linalg.lu(H)
  return (L,U)

In [94]:
get_Hillbert_LU(create_hillbert(4)[0])

(array([[ 1.        ,  0.        ,  0.        ,  0.        ],
        [ 0.33333333,  1.        ,  0.        ,  0.        ],
        [ 0.5       ,  1.        ,  1.        ,  0.        ],
        [ 0.25      ,  0.9       , -0.6       ,  1.        ]]),
 array([[ 1.00000000e+00,  5.00000000e-01,  3.33333333e-01,
          2.50000000e-01],
        [ 0.00000000e+00,  8.33333333e-02,  8.88888889e-02,
          8.33333333e-02],
        [ 0.00000000e+00,  0.00000000e+00, -5.55555556e-03,
         -8.33333333e-03],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          3.57142857e-04]]))

In [95]:
solutions = []
condition_number = []
relative_error = []
relative_residual = []

for item in n_values:
  #create the appropriate matrix
  H = create_hillbert(item)
  #append the condition number with the infinite norm
  condition_number.append(np.linalg.cond(H[0], np.inf))
  L,U = get_Hillbert_LU(H[0])
  #solve for x_approximate as instructed
  x_approximate = np.linalg.solve(U, np.linalg.solve(L,H[1]))
  solutions.append(x_approximate)
  #exact solution is vector of 1's with item rows (item will be 5, 10, 15, 20)
  exact_solution = np.ones((item,1))

  #find relative error and relative residual with infinity norm
  #since the exact solution is a column vector of 1's its infinity norm (largest row sum) = 1
  rel_error = (np.linalg.norm(exact_solution-x_approximate), np.inf)
  relative_error.append(rel_error)
  #np.dot is used for matrix multiplication
  rel_residual = (np.linalg.norm((H[1] - (np.dot(H[0],x_approximate))), np.inf))/(np.linalg.norm(H[1], np.inf))
  relative_residual.append(rel_residual)

In [96]:
solutions

[array([   765.08730159, -13946.61904771,  59309.33333372, -88737.88888949,
         43126.0000003 ]),
 array([-7.47637652e+06,  6.62160908e+08, -1.43649053e+10,  1.32457162e+11,
        -6.38976646e+11,  1.77280929e+12, -2.93100365e+12,  2.85084694e+12,
        -1.50496839e+12,  3.32551700e+11]),
 array([-5.93287616e+08,  8.97669686e+10, -3.43019643e+12,  5.79378081e+13,
        -5.39252868e+14,  3.09845637e+15, -1.17275918e+16,  3.03514297e+16,
        -5.47088243e+16,  6.88631022e+16, -5.96852691e+16,  3.43406111e+16,
        -1.21616025e+16,  2.24972403e+15, -1.35379633e+14]),
 array([ 6.98630790e+09, -1.25493292e+12,  5.56125612e+13, -1.06261364e+15,
         1.08660751e+16, -6.60251748e+16,  2.49907659e+17, -5.88694819e+17,
         7.93399102e+17, -3.66738831e+17, -5.29717470e+17,  8.24258913e+17,
        -9.09049345e+16, -6.35320311e+17,  6.32670430e+17, -4.21177302e+17,
         3.67293981e+17, -2.57220557e+17,  8.85338058e+16, -1.01223121e+16])]

In [97]:
results = pd.DataFrame(columns = ['n', 'relative error', 'condition number', 'relative residual'])
results['n'] = n_values
results['condition number'] = condition_number
results['relative error'] = relative_error
results['relative residual'] = relative_residual

In [98]:
results

,n,relative error,condition number,relative residual
0,5,"(259296.68659370445, inf)",9.436560e+05,0.156413
1,10,"(15054078116384.676, inf)",3.535685e+13,0.258287
2,15,"(4.532948654324339e+17, inf)",1.041727e+18,0.734004
3,20,"(8.168283284632813e+18, inf)",6.008377e+18,6.390855
